# Segmenting and Clustering Neighborhoods in Toronto

#### In this project I explored Toronto, a city in Canada. This project examines the near by Venues
#### of every Neighborhood of Toronto and then Clusters the Similar Neighborhoods togeter.
#### This project helps to find a suitable Neighborhood of your kind  to live in this city.


## Forming the Data Frame

In [1]:
# importing required libraries
import pandas as pd #for data cleaning
pd.set_option('display.max_columns', None) #displays maximum columns
pd.set_option('display.max_rows', None) #display maximum rows
import requests #for importing data from a url
import numpy as np #for data cleaning and powerful math operations
from bs4 import BeautifulSoup as bs #for data scrapping
from geopy.geocoders import Nominatim #for getting location 
import folium #for forming maps
from sklearn.cluster import KMeans #for machine learning clustring
import matplotlib.cm as cm #for handeling colors
import matplotlib.colors as colors #for giving colors to clusters




#### Web scraping to get Data Frame

In [2]:
#getting the html file from url
r = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")


In [3]:
#scraping data frame from html file
soup = bs(r.content)
table = soup.find_all("table")
df = pd.read_html(str(table))[0]

In [4]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Cleaning the data frame

In [5]:
#replace all not assigned values with NaN
df.replace("Not assigned", np.nan, inplace = True)

In [6]:
# droping all NaN values from Borough
df.dropna(subset=["Borough"], inplace =True)

In [7]:
df.sort_values(by="Postal Code", inplace = True)

In [8]:
# Reseting index
df.reset_index(inplace = True)

In [9]:
#dropping old index
df.drop(["index"], axis = 1, inplace = True)

In [10]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
#getting shape of final Data Frame
df.shape

(103, 3)

#### importing and merging longitude and latitudes

In [12]:
lat_long = pd.read_csv("Geospatial_Coordinates.csv")

In [13]:
lat_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
#dropping Postal code
lat_long.drop(["Postal Code"], axis = 1, inplace = True)

In [15]:
df_new = pd.concat([df,lat_long], axis = 1)

In [16]:
df_new.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Getting location of Toronto

In [17]:
address = "Toronto,Ontario"
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
lat = location.latitude
lng = location.longitude
lat_lng_coords = (lat,lng)


In [18]:
lat_lng_coords

(43.6534817, -79.3839347)

#### Forming map of toronto 

In [19]:
map_toronto = folium.Map(location=[lat,lng], zoom_start=10)
for lati, longi, borough, neighbour in zip(df_new["Latitude"],df_new["Longitude"],df_new["Borough"],df_new["Neighbourhood"]):
    label = "{}, {}".format(borough, neighbour)
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lati, longi],
        popup = label,
        radius = 7,
        color = "yellow",
        fill = True,
        fill_color = "#3186cc",
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto)
    
map_toronto
        
        
        
    

#### Cleaning neighborhood data to just Toronto

In [20]:
toronto_data = []
for i in df_new["Borough"]:
    if "Toronto" in i:
        toronto_data.append(i)
    else:
        toronto_data.append(np.nan)
    

In [21]:
toronto_data = df_new[df_new["Borough"] == toronto_data]

In [22]:
toronto_data.reset_index(inplace=True)

In [23]:
toronto_data.head()

,index,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923
4,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


#### Forming Toronto map with just Neighborhood containing Toronto

In [24]:
map_toronto = folium.Map(location=[lat,lng], zoom_start=10)
for lati, longi, borough, neighbour in zip(toronto_data["Latitude"],toronto_data["Longitude"],toronto_data["Borough"],toronto_data["Neighbourhood"]):
    label = "{}, {}".format(borough, neighbour)
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lati, longi],
        popup = label,
        radius = 7,
        color = "yellow",
        fill = True,
        fill_color = "#3186cc",
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto)
    
map_toronto

## Foursquare
#### making arrangement for API calls

In [25]:
CLIENT_ID = 'JMTGCDC0QSE4QZP32C1JHCRHTQDIZZYCVZC0DLTPK0PMURZX' # your Foursquare ID
CLIENT_SECRET = 'IVLDBLT0VZUVM5SVYZSKSW4RV0MDTHXZX31E3CLSLYOMKK5E' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JMTGCDC0QSE4QZP32C1JHCRHTQDIZZYCVZC0DLTPK0PMURZX
CLIENT_SECRET:IVLDBLT0VZUVM5SVYZSKSW4RV0MDTHXZX31E3CLSLYOMKK5E


In [26]:
radius = 500
limit = 100
url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,limit)
results = requests.get(url).json()["response"]["groups"][0]["items"]

In [27]:
results


[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '5227bb01498e17bf485e6202',
   'name': 'Downtown Toronto',
   'location': {'lat': 43.65323167517444,
    'lng': -79.38529600606677,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.65323167517444,
      'lng': -79.38529600606677}],
    'distance': 113,
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['Toronto ON', 'Canada']},
   'categories': [{'id': '4f2a25ac4b909258e854f55f',
     'name': 'Neighborhood',
     'pluralName': 'Neighborhoods',
     'shortName': 'Neighborhood',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neighborhood_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-5227bb01498e17bf485e6202-0'},
 {'reasons': {'count': 0,
   'items': [{'summar

### Getting Nearby Venues of Neighborhoods

In [28]:
def neighbor_toronto(names,latitudes,longitudes,radius=500):
    venues_list = []
    for name, lat, long in zip(names, latitudes, longitudes):
        
        url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,long,radius,limit)
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        
        venues_list.append([(
        name,
        lat,
        long,
        v["venue"]["name"],
        v["venue"]["location"]["lat"],
        v["venue"]["location"]["lng"],
        v["venue"]["categories"][0]["name"]) for v in results])
        
        nearby_venues = pd.DataFrame([item for venues_list in venues_list for item in venues_list])
        nearby_venues.columns = ["neighbourhood",
                                "neighbour latitude",
                                "neighbour longitude",
                                "venue name",
                                "venue latitude",
                                "venue longitude",
                                "venue category"]
        
    return(nearby_venues)
        
        

In [29]:
toronto_venues = neighbor_toronto(toronto_data["Neighbourhood"],toronto_data["Latitude"],toronto_data["Longitude"])

In [30]:
toronto_venues.head()

,neighbourhood,neighbour latitude,neighbour longitude,venue name,venue latitude,venue longitude,venue category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [31]:
toronto_venues.shape

(1647, 7)

In [32]:
#grouping data by neighbourhood to get total venues in each neighbourhood
toronto_venues.groupby("neighbourhood").count()

,neighbour latitude,neighbour longitude,venue name,venue latitude,venue longitude,venue category
neighbourhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",25,25,25,25,25,25
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",19,19,19,19,19,19
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,67,67,67,67,67,67
Christie,16,16,16,16,16,16
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,36,36,36,36,36,36


#### Getting total number of venue categories

In [33]:
dtgs = toronto_venues["venue category"].unique()

In [34]:
dtgs = pd.DataFrame(dtgs)
print("there are{} unique categories".format(dtgs.count()))

there are0    234
dtype: int64 unique categories


## Analysing Neighborhood Data

#### displaying each venue in couloms and each neighborhood in rows

In [35]:
toronto_onehot = pd.get_dummies(toronto_venues["venue category"],prefix="",prefix_sep="")

In [36]:
toronto_onehot.head()

,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [37]:
toronto_onehot.shape

(1647, 234)

In [38]:
toronto_onehot["neighbourhood"] = toronto_venues["neighbourhood"]
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [39]:
toronto_onehot.head()

,neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### Grouping by neighborhood and taking mean of each venue in a neighborhood

In [40]:
toronto_grouped = toronto_onehot.groupby("neighbourhood").mean().reset_index()

In [41]:
toronto_grouped.head()

,neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,0.017544,0.000000,0.0,0.0,0.000000,0.017544,0.0,0.017544,0.035088,0.0,0.000000,0.0,0.017544,0.017544,0.0,0.035088,0.0,0.0,0.017544,0.0,0.0000,0.0,0.0000,0.0,0.017544,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.035088,0.0,0.0,0.0,0.0,0.035088,0.000000,0.0,0.0,0.00,0.0,0.035088,0.105263,0.0,0.0,0.0,0.0,0.0,0.017544,0.000000,0.017544,0.00,0.0,0.0,0.017544,0.0,0.0,0.0,0.0,0.017544,0.000000,0.0,0.017544,0.000000,0.0,0.0,0.0,0.000000,0.0,0.017544,0.0,0.0,0.0,0.000000,0.035088,0.000000,0.0,0.0,0.017544,0.0,0.0,0.0,0.0,0.017544,0.017544,0.0,0.0,0.0,0.000000,0.0,0.000000

In [42]:
toronto_grouped.shape

(39, 235)

In [43]:
#function for returning top most common venues in a neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Making a data frame with neighborhoods and its top most common venues

In [44]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['neighbourhood'] = toronto_grouped['neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Bakery,Cocktail Bar,Beer Bar,Farmers Market,Cheese Shop,Seafood Restaurant,Café,Eastern European Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Yoga Studio,Bakery,Breakfast Spot,Coffee Shop,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Garden Center,Skate Park,Restaurant,Recording Studio,Pizza Place,Park,Garden,Spa
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boutique,Airport,Airport Food Court,Sculpture Garden,Harbor / Marina,Bar,Rental Car Location
4,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Japanese Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Department Store,Miscellaneous Shop


## Clustering

In [45]:
toronto_clusturing = toronto_grouped.drop("neighbourhood", axis = 1)

In [46]:
toronto_clusturing.head()

,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0.0,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,0.017544,0.000000,0.0,0.0,0.000000,0.017544,0.0,0.017544,0.035088,0.0,0.000000,0.0,0.017544,0.017544,0.0,0.035088,0.0,0.0,0.017544,0.0,0.0000,0.0,0.0000,0.0,0.017544,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.035088,0.0,0.0,0.0,0.0,0.035088,0.000000,0.0,0.0,0.00,0.0,0.035088,0.105263,0.0,0.0,0.0,0.0,0.0,0.017544,0.000000,0.017544,0.00,0.0,0.0,0.017544,0.0,0.0,0.0,0.0,0.017544,0.000000,0.0,0.017544,0.000000,0.0,0.0,0.0,0.000000,0.0,0.017544,0.0,0.0,0.0,0.000000,0.035088,0.000000,0.0,0.0,0.017544,0.0,0.0,0.0,0.0,0.017544,0.017544,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0

### Fitting the data in K-Means clustering and forming 5 clusters

In [47]:
kclusters = 5
toronto_kmeans = KMeans(n_clusters = kclusters,random_state=0)
toronto_kmeans.fit(toronto_clusturing)
toronto_kmeans.labels_

array([2, 2, 0, 2, 2, 0, 2, 2, 0, 2, 0, 2, 1, 2, 2, 0, 0, 2, 0, 2, 1, 2,
       2, 2, 2, 2, 1, 4, 2, 2, 0, 2, 2, 2, 0, 3, 2, 2, 0])

In [48]:
#inserting clusters into data frame
neighborhoods_venues_sorted.insert(0,"Clusters",toronto_kmeans.labels_)

In [49]:
neighborhoods_venues_sorted.head()

,Clusters,neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,Berczy Park,Coffee Shop,Restaurant,Bakery,Cocktail Bar,Beer Bar,Farmers Market,Cheese Shop,Seafood Restaurant,Café,Eastern European Restaurant
1,2,"Brockton, Parkdale Village, Exhibition Place",Café,Yoga Studio,Bakery,Breakfast Spot,Coffee Shop,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue
2,0,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Garden Center,Skate Park,Restaurant,Recording Studio,Pizza Place,Park,Garden,Spa
3,2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boutique,Airport,Airport Food Court,Sculpture Garden,Harbor / Marina,Bar,Rental Car Location
4,2,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Japanese Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Department Store,Miscellaneous Shop


#### Merging clusters with Final Data frame

In [50]:
toronto_data.head()

,index,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923
4,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [51]:
toronto_merged = toronto_data

In [52]:
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index("neighbourhood"),on="Neighbourhood")

In [53]:
toronto_merged.head()

,index,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Clusters,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Trail,Neighborhood,Health Food Store,Pub,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio
1,41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Restaurant,Yoga Studio,Liquor Store,Spa,Juice Bar
2,42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Park,Brewery,Movie Theater,Ice Cream Shop,Sushi Restaurant,Fast Food Restaurant,Restaurant,Pub,Italian Restaurant,Board Shop
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Café,Coffee Shop,American Restaurant,Bakery,Brewery,Gastropub,Yoga Studio,Fish Market,Pet Store,Park
4,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Dim Sum Restaurant,Park,Bus Line,Swim School,Yoga Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


### Forming Map of Toronto With Clusters of differnt Neighborhood kind 

In [54]:
map_clusters = folium.Map(location=[lat,lng],zoom_start=11)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for latt, lngg, name, cluster in zip(toronto_merged["Latitude"],toronto_merged["Longitude"],toronto_merged["Neighbourhood"],toronto_merged["Clusters"]):
    label = folium.Popup(str(name) + ": cluster:" + str(cluster),parse_html=True)
    folium.CircleMarker(
        [latt,lngg],
        radius=7,
        popup=label,
        color=rainbow[cluster-1],
        fill= True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
        
map_clusters  

## Exploring All 5 clusters

### Cluster 1

In [55]:
toronto_merged.loc[toronto_merged["Clusters"] == 0,toronto_merged.columns[[3]+list(range(7,17))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"India Bazaar, The Beaches West",Park,Brewery,Movie Theater,Ice Cream Shop,Sushi Restaurant,Fast Food Restaurant,Restaurant,Pub,Italian Restaurant,Board Shop
4,Lawrence Park,Dim Sum Restaurant,Park,Bus Line,Swim School,Yoga Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
7,Davisville,Pizza Place,Dessert Shop,Sandwich Place,Sushi Restaurant,Coffee Shop,Italian Restaurant,Gym,Toy / Game Store,Café,Pharmacy
11,"St. James Town, Cabbagetown",Coffee Shop,Pizza Place,Italian Restaurant,Park,Pharmacy,Bakery,Restaurant,Café,Pub,General Entertainment
24,"The Annex, North Midtown, Yorkville",Café,Sandwich Place,Coffee Shop,Pizza Place,Pub,Donut Shop,BBQ Joint,Indian Restaurant,History Museum,Pharmacy
25,"University of Toronto, Harbord",Café,Restaurant,Bar,Bookstore,Sandwich Place,Japanese Restaurant,Bakery,Chinese Restaurant,Dessert Shop,Pub
30,Christie,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Diner,Restaurant,Candy Store,Baby Store,Coffee Shop
31,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Grocery Store,Bar,Supermarket,Middle Eastern Restaurant,Café,Music Venue,Pizza Place,Brewery
34,"High Park, The Junction South",Café,Mexican Restaurant,Thai Restaurant,Grocery Store,Furniture / Home Store,Fast Food Restaurant,Bookstore,Cajun / Creole Restaurant,Italian Restaurant,Speakeasy
38,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Garden Center,Skate Park,Restaurant,Recording Studio,Pizza Place,Park,Garden,Spa


### Cluster 2

In [56]:
toronto_merged.loc[toronto_merged["Clusters"] == 1,toronto_merged.columns[[3]+list(range(7,17))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Moore Park, Summerhill East",Restaurant,Park,Trail,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
10,Rosedale,Park,Playground,Trail,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
23,"Forest Hill North & West, Forest Hill Road Park",Park,Jewelry Store,Trail,Sushi Restaurant,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


### Cluster 3

In [57]:
toronto_merged.loc[toronto_merged["Clusters"] == 2,toronto_merged.columns[[3]+list(range(7,17))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Restaurant,Yoga Studio,Liquor Store,Spa,Juice Bar
3,Studio District,Café,Coffee Shop,American Restaurant,Bakery,Brewery,Gastropub,Yoga Studio,Fish Market,Pet Store,Park
5,Davisville North,Gym,Hotel,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Park,General Travel,Gluten-free Restaurant,Eastern European Restaurant
6,"North Toronto West, Lawrence Park",Clothing Store,Coffee Shop,Yoga Studio,Mexican Restaurant,Shoe Store,Seafood Restaurant,Salon / Barbershop,Restaurant,Pet Store,Park
9,"Summerhill West, Rathnelly, South Hill, Forest...",Pub,Coffee Shop,Bagel Shop,Pizza Place,Supermarket,Sushi Restaurant,Fried Chicken Joint,Sports Bar,Restaurant,Bank
12,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Pub,Men's Store,Mediterranean Restaurant,Hotel,Dance Studio
13,"Regent Park, Harbourfront",Coffee Shop,Pub,Bakery,Park,Café,Theater,Breakfast Spot,Health Food Store,Historic Site,Hotel
14,"Garden District, Ryerson",Clothing Store,Coffee Shop,Café,Japanese Restaurant,Cosmetics Shop,Bubble Tea Shop,Italian Restaurant,Electronics Store,Pizza Place,Fast Food Restaurant
15,St. James Town,Café,Coffee Shop,Clothing Store,Restaurant,Cosmetics Shop,Cocktail Bar,American Restaurant,Breakfast Spot,Park,Moroccan Restaurant
16,Berczy Park,Coffee Shop,Restaurant,Bakery,Cocktail Bar,Beer Bar,Farmers Market,Cheese Shop,Seafood Restaurant,Café,Eastern European Restaurant


### Cluster 4

In [58]:
toronto_merged.loc[toronto_merged["Clusters"] == 3,toronto_merged.columns[[3]+list(range(7,17))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,Trail,Neighborhood,Health Food Store,Pub,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio


### Cluster 5

In [59]:
toronto_merged.loc[toronto_merged["Clusters"] == 4,toronto_merged.columns[[3]+list(range(7,17))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Roselawn,Home Service,Music Venue,Garden,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
